In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [7]:
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16

print("Import OK")

Import OK


In [42]:
model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = model.output
x = GlobalAveragePooling2D()(x)
#x = Dense(1024, activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
#x = Dense(1024, activation='relu')(x) #dense layer 2
#x = Dropout(0.25)(x)
x = Dense(512, activation='relu')(x) #dense layer 3
#x = Dense(256, activation='relu')(x) #dense layer 4
x = Dropout(0.4)(x)
x = Dense(128, activation='relu')(x) #dense layer 5
x = Dropout(0.25)(x)
#x = Dense(64, activation='relu')(x) #dense layer 6
#x = Dense(32, activation='relu')(x)
preds = Dense(13, activation='softmax')(x) #final layer with softmax activation

model2 = Model(inputs=model.input, outputs=preds)
for layer in model2.layers[:20]:
    layer.trainable=False
for layer in model2.layers[20:]:
    layer.trainable=True
    
model2.summary()

Model: "model_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0  

In [43]:
from keras.applications.vgg16 import preprocess_input, decode_predictions
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/', # this is where you specify the path to the main data folder
                                                 target_size=(224, 224),
                                                 #color_mode='grayscale',
                                                 batch_size=64,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 2819 images belonging to 13 classes.


In [44]:
model2.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train=train_generator.n//train_generator.batch_size
model2.fit_generator(generator=train_generator, steps_per_epoch=step_size_train, epochs=20)

Epoch 1/20
44/44 [==============================] - 9s 194ms/step - loss: 1.5620 - accuracy: 0.5731
Epoch 2/20
44/44 [==============================] - 9s 201ms/step - loss: 0.5245 - accuracy: 0.8261
Epoch 3/20
44/44 [==============================] - 9s 200ms/step - loss: 0.3945 - accuracy: 0.8701
Epoch 4/20
44/44 [==============================] - 9s 211ms/step - loss: 0.3355 - accuracy: 0.8922
Epoch 5/20
44/44 [==============================] - 9s 197ms/step - loss: 0.2899 - accuracy: 0.8995
Epoch 6/20
44/44 [==============================] - 9s 197ms/step - loss: 0.2636 - accuracy: 0.9103
Epoch 7/20
44/44 [==============================] - 9s 198ms/step - loss: 0.2105 - accuracy: 0.9245
Epoch 8/20
44/44 [==============================] - 9s 202ms/step - loss: 0.1578 - accuracy: 0.9482
Epoch 9/20
44/44 [==============================] - 9s 198ms/step - loss: 0.2095 - accuracy: 0.9325
Epoch 10/20
44/44 [==============================] - 9s 199ms/step - loss: 0.1511 - accuracy: 0.9488

In [45]:
import cv2

test_data_name = []
test_data = []

for dirname, _, filenames in os.walk('/kaggle/input/cs-ioc5008-hw1/dataset/dataset/test'):
    for filename in filenames:
        test_data_name.append(filename)
        image = cv2.imread(os.path.join(dirname, filename))
        test_data.append(image)
        
print("Number of test data =", len(test_data))

Number of test data = 1040


In [46]:
def processing(image_data):
    #img = [data[123], data[1054], data[156]]
    #print("Original size:", img[1].shape)
    #print(training_label[1054])
    #original = img[1]
    #display_one(original)
    
    height = 224
    width = 224
    dim = (width, height)
    res_img = []
    
    for i in range(len(image_data)):
        res = cv2.resize(image_data[i], dim, interpolation=cv2.INTER_LINEAR)
        res_img.append(res)
    
    return res_img

test_data_reshape = processing(test_data)

print(test_data_reshape[0].shape)

(224, 224, 3)


In [47]:
for i in range(len(test_data_reshape)):
    test_data_reshape[i] = preprocess_input(test_data_reshape[i])

In [48]:
#for i in range(len(test_data_reshape)):
#    test_data_reshape[i] = (test_data_reshape[i] - 128.0) / 128.0
    
for i in range(len(test_data_reshape)):
    test_data_reshape[i] = test_data_reshape[i].reshape(-1, 224, 224, 3)
    
scene_detail = ["bedroom", "coast", "forest", "highway", "insidecity", "kitchen", "livingroom", "mountain", "office", "opencountry", "street", "suburb", "tallbuilding"]

print("id,label")

for i in range(len(test_data_reshape)):
    name_arr = test_data_name[i].split('.')
    name = name_arr[0]
    answer = model2.predict(test_data_reshape[i])
    print(name+","+scene_detail[np.argmax(answer)])


id,label
image_0552,mountain
image_0340,coast
image_0320,highway
image_0150,opencountry
image_0602,highway
image_0640,street
image_0563,highway
image_0030,street
image_0601,office
image_0994,suburb
image_1027,mountain
image_0450,forest
image_0312,forest
image_0230,office
image_0305,suburb
image_0322,office
image_0427,insidecity
image_0183,livingroom
image_0681,coast
image_0979,opencountry
image_0702,highway
image_0383,bedroom
image_0648,livingroom
image_0003,bedroom
image_0801,mountain
image_0754,street
image_0332,office
image_1026,suburb
image_0935,opencountry
image_0663,livingroom
image_0722,kitchen
image_0926,office
image_0985,kitchen
image_0101,coast
image_0131,suburb
image_0221,opencountry
image_0915,forest
image_0282,forest
image_0149,forest
image_0420,street
image_0273,coast
image_0465,insidecity
image_0116,livingroom
image_1016,livingroom
image_0251,suburb
image_0775,street
image_0075,coast
image_0938,mountain
image_0008,mountain
image_0368,office
image_0554,highway
image_0170,

image_0655,insidecity
image_0876,highway
image_0192,mountain
image_0919,tallbuilding
image_0265,insidecity
image_0162,highway
image_0062,suburb
image_0956,kitchen
image_0619,coast
image_0014,insidecity
image_0786,livingroom
image_0687,livingroom
image_0933,coast
image_0524,office
image_0813,street
image_0337,insidecity
image_0434,suburb
image_0268,mountain
image_0364,forest
image_0591,street
image_0203,tallbuilding
image_0698,street
image_0995,livingroom
image_0864,suburb
image_0120,highway
image_0572,tallbuilding
image_0575,bedroom
image_0881,office
image_0582,office
image_0173,opencountry
image_0791,opencountry
image_0036,street
image_0860,livingroom
image_0617,forest
image_0021,mountain
image_0647,insidecity
image_0710,tallbuilding
image_0828,livingroom
image_0669,street
image_0095,livingroom
image_0470,coast
image_0594,forest
image_0411,opencountry
image_0622,insidecity
image_0851,kitchen
image_0555,bedroom
image_0856,kitchen
image_0892,office
image_0841,livingroom
image_0164,coast

image_0774,kitchen
image_0625,insidecity
image_0547,tallbuilding
image_0521,opencountry
image_0729,opencountry
image_0092,kitchen
image_0206,opencountry
image_0278,street
image_0064,coast
image_0345,livingroom
image_0651,tallbuilding
image_0976,suburb
image_0227,bedroom
image_0604,suburb
image_0109,office
image_0533,mountain
image_0964,bedroom
image_0148,street
image_0526,opencountry
image_0825,street
image_0945,livingroom
image_0650,mountain
image_0543,suburb
image_0029,forest
image_0152,mountain
image_0009,street
image_0247,mountain
image_0690,forest
image_1031,coast
image_0493,bedroom
image_0168,coast
image_0576,mountain
image_0198,tallbuilding
image_0955,coast
image_0536,bedroom
image_0343,tallbuilding
image_0145,forest
image_1014,bedroom
image_0158,bedroom
image_0779,opencountry
image_0540,street
image_0354,opencountry
image_0442,livingroom
image_0229,suburb
image_0769,office
image_0750,coast
image_0632,opencountry
image_0732,insidecity
image_0900,bedroom
image_0895,office
image_0